In [9]:
import pandas as pd
import sys
import os.path
import numpy as np
import cProfile
import pstats
import matplotlib.pyplot as plt
from multiprocessing import Pool
import time

# import bokeh.plotting
# from bokeh.plotting import figure, output_notebook, show, gridplot, reset_output

import spykshrk.realtime.binary_record as bin_rec
import spykshrk.realtime.binary_record_cy as bin_rec_cy

pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 180)

# output_notebook()

In [10]:
start_time = time.monotonic()

#rec_dir = '/opt/data36/daliu/realtime/spykshrk/bond_test'
rec_dir = '/opt/data36/daliu/realtime/spykshrk/timing'
# rec_dir = '/home/daliu/output/test'
mpi_ranks = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
file_prefix = 'test_animal'
manager_label = 'state'
file_postfix = 'bin_rec'

p = Pool(20)

allrecs = {}

bin_panda_convert_list = []
for mpi_rank in mpi_ranks:
    binrec = bin_rec.BinaryRecordsFileReader(save_dir=rec_dir, file_prefix=file_prefix,
                                             mpi_rank=mpi_rank,
                                             manager_label=manager_label,
                                             file_postfix=file_postfix, filemeta_as_col=True)

    bin_panda_convert_list.append(binrec)
    
panda_list = p.map(bin_rec.reader_multiprocessing_helper, bin_panda_convert_list)

stop_time = time.monotonic()
print('Runtime {:} mins.'.format((stop_time - start_time)/60))


In [11]:
start_time = time.monotonic()

for rec_pandas in panda_list:
    for rec_id, rec_pd in rec_pandas.items():
        existing_rec_pd = allrecs.setdefault(rec_id, pd.DataFrame())
        allrecs[rec_id] = existing_rec_pd.append(rec_pd, ignore_index=True)
    
for rec_id, rec_pd in allrecs.items():
    if 'timestamp' in rec_pd.columns:
        rec_pd.sort_values(['timestamp']).reset_index(drop=True)
        
stop_time = time.monotonic()
print('Runtime {:} mins.'.format((stop_time - start_time)/60))


In [13]:
allrecs[2]

In [5]:
allrecs[1].dtypes

In [10]:
allrecs.keys()

In [24]:
spike_dec = allrecs[4]
spike_dec

In [25]:
spike_dec.loc[942, 'x0':'x149'].values

In [28]:
p1 = plt.plot(spike_dec.loc[904, 'x0':'x149'].values)

plt.show(p1)

In [43]:
plt.figure(figsize=[20,10])
p1 = plt.plot(np.argmax(spike_dec.loc[:, 'x0':'x149'].values, axis=1))

plt.show(p1)

In [ ]:

plt.figure(figsize=[25,10])
im1 = plt.imshow(np.transpose(decode_values), aspect='auto')
plt.show(im1)